In [317]:
import numpy as np
import pandas as pd
import scipy
import math
import statsmodels
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from WindPy import w
import calendar
import numpy as np
import scipy.stats as stats
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
%matplotlib inline

# 1. 股票

In [318]:
symbol = "881001.WI"

### 某一个时间点的财务类指标

In [375]:
factors = ["mkt_cap_ard", # 总市值
           "pe_ttm", # 市盈率
           "pcf_ocf_ttm", # 市现率
           "dividendyield2", # 股息率
           "profit_ttm", # 净利润
           "or_ttm", # 营业收入
           "ps_ttm" # 市销率
          ]

In [376]:
raw_data = w.wss(symbol, factors, "tradeDate=%s"%("20161220"))

In [377]:
raw_data

.ErrorCode=0
.Codes=[881001.WI]
.Fields=[MKT_CAP_ARD,PE_TTM,PCF_OCF_TTM,DIVIDENDYIELD2,PROFIT_TTM,OR_TTM,PS_TTM]
.Times=[20170215 08:45:52]
.Data=[[5.53584423133e+13],[21.3279],[9.0018],[1.3199],[nan],[nan],[1.8015]]

In [378]:
dic = {}
for data, field in zip(raw_data.Data, raw_data.Fields):
    dic[str(field.lower())] = data

In [379]:
dic

{'dividendyield2': [1.3199],
 'mkt_cap_ard': [55358442313309.0],
 'or_ttm': [nan],
 'pcf_ocf_ttm': [9.0018],
 'pe_ttm': [21.3279],
 'profit_ttm': [nan],
 'ps_ttm': [1.8015]}

# 获取历史财务数据

In [438]:
current_date = datetime.datetime(2016, 12, 20)
end_date = datetime.datetime(2016, 12, 31)
dic = {}
dates = []
while current_date <= end_date:
    print current_date.strftime("%Y%m%d")
    raw_data = w.wss(symbol, factors, "tradeDate=%s"%(current_date.strftime("%Y%m%d")))
    for data, field in zip(raw_data.Data, raw_data.Fields):
        if not dic.has_key(str(field.lower())):
            dic[str(field.lower())] = data
        else:
            dic[str(field.lower())].append(data[0])
    dates.append(current_date)
    current_date = current_date + datetime.timedelta(1)

20161220
20161221
20161222
20161223
20161224
20161225
20161226
20161227
20161228
20161229
20161230
20161231


In [439]:
df = pd.DataFrame(dic, index=dates)

In [440]:
df["date"] = dates

In [441]:
df.head()

,dividendyield2,mkt_cap_ard,or_ttm,pcf_ocf_ttm,pe_ttm,profit_ttm,ps_ttm,date
2016-12-20,1.3199,5.535844e+13,NaN,9.0018,21.3279,NaN,1.8015,2016-12-20
2016-12-21,1.3069,5.591413e+13,NaN,9.0919,21.5406,NaN,1.8195,2016-12-21
2016-12-22,1.3067,5.592158e+13,NaN,9.0931,21.5429,NaN,1.8197,2016-12-22
2016-12-23,1.3183,5.544383e+13,NaN,9.0150,21.3566,NaN,1.8040,2016-12-23
2016-12-24,1.3184,5.544384e+13,NaN,9.0150,21.3567,NaN,1.8040,2016-12-24


In [442]:
df.to_csv("%s.csv"%(symbol))